In [1]:
import numpy as np
import pandas as pd
from skimage.feature import hog
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score
import joblib
from collections import Counter
import cv2

height, width = 10, 10

In [2]:
data = pd.read_csv(f"../archive/ascii_character_classification_{height}_x_{width}.csv", header=0).sample(frac=.05)
label_counts = Counter(data.iloc[:, 0])
print(label_counts)

Counter({0: 2472, 80: 275, 28: 275, 60: 270, 30: 269, 10: 266, 32: 261, 2: 260, 34: 260, 87: 258, 59: 255, 26: 255, 51: 255, 88: 254, 56: 252, 33: 252, 54: 251, 70: 250, 93: 249, 62: 249, 94: 248, 61: 248, 43: 248, 23: 248, 16: 247, 17: 246, 39: 244, 69: 243, 71: 243, 35: 243, 12: 242, 4: 242, 92: 242, 53: 242, 58: 242, 75: 241, 79: 241, 24: 241, 47: 241, 78: 240, 3: 240, 11: 240, 8: 240, 42: 240, 86: 239, 22: 239, 90: 238, 81: 238, 82: 237, 91: 237, 31: 237, 50: 237, 83: 236, 52: 236, 85: 235, 77: 234, 41: 234, 6: 233, 76: 233, 48: 233, 67: 233, 13: 233, 63: 232, 72: 231, 29: 230, 18: 230, 21: 230, 74: 229, 68: 229, 84: 228, 1: 227, 49: 227, 37: 226, 14: 226, 66: 226, 20: 225, 45: 224, 27: 223, 25: 223, 7: 222, 46: 221, 89: 221, 40: 220, 9: 219, 57: 217, 44: 217, 65: 217, 5: 216, 64: 216, 55: 216, 36: 215, 95: 214, 38: 213, 15: 211, 73: 210, 19: 207})


In [3]:
X = data.iloc[:, 1:].astype("float64")
y = data.iloc[:, 0].astype("float64")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
# K-NN Classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
joblib.dump(knn, f'../artifacts/knn_model_{height}_x_{width}.pkl')

['../artifacts/knn_model_10_x_10.pkl']

In [5]:
y_pred = knn.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred)
y_pred = knn.predict(X_test)
print("test Shape:", X_test.shape)

test_accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print(f"Train Accuracy: {train_accuracy*100:.4f}%")
print(f"Test Accuracy: {test_accuracy*100:.4f}%")
print(f"F1 Score: {f1*100:.4f}%")
print(f"Recall: {recall*100:.4f}%")

test Shape: (5000, 100)
Train Accuracy: 96.8900%
Test Accuracy: 95.6800%
F1 Score: 94.9894%
Recall: 95.6800%


In [6]:
def extract_hog_features(images):
    hog_features = []
    for image in images:
        image_reshaped = image.reshape((height, width))
        features = hog(image_reshaped, pixels_per_cell=(2, 2), cells_per_block=(1, 1), feature_vector=True)
        hog_features.append(features)
    return np.array(hog_features)

X_hog = extract_hog_features(np.array(X))
X_train, X_test, y_train, y_test = train_test_split(X_hog, y, test_size=0.2, random_state=42)

# %%
# K-NN Classifier with HOG features
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
joblib.dump(knn, f'../artifacts/knn_model_hog_{height}_x_{width}.pkl')

# %%
y_pred = knn.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred)
y_pred = knn.predict(X_test)
print("test Shape:", X_test.shape)

test_accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print(f"Train Accuracy: {train_accuracy*100:.4f}%")
print(f"Test Accuracy: {test_accuracy*100:.4f}%")
print(f"F1 Score: {f1*100:.4f}%")
print(f"Recall: {recall*100:.4f}%")

test Shape: (5000, 225)
Train Accuracy: 91.8150%
Test Accuracy: 85.1200%
F1 Score: 84.6757%
Recall: 85.1200%


In [ ]:
def extract_sift_features(images):
    sift = cv2.SIFT_create()
    sift_features = []
    
    for image in images:
        image_reshaped = image.reshape((height, width)).astype(np.uint8)
        keypoints, descriptors = sift.detectAndCompute(image_reshaped, None)
        
        # If no keypoints are found, use a zero array of the same length as a typical descriptor
        if descriptors is None:
            descriptors = np.zeros((1, sift.descriptorSize()), dtype=np.float32)
        
        # Flatten descriptors and use them as features
        features = descriptors.flatten()
        sift_features.append(features)
    
    return np.array(sift_features)

# Extract SIFT features
X_sift = extract_sift_features(np.array(X))

# %%
X_sift = extract_sift_features(np.array(X))

# Since the number of features might vary, we need to ensure consistent feature vector size
# Here, we'll pad with zeros to the maximum descriptor length found
max_len = max(len(f) for f in X_sift)
X_sift = np.array([np.pad(f, (0, max_len - len(f)), 'constant') for f in X_sift])

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_sift, y, test_size=0.2, random_state=42)

# %%
# K-NN Classifier with SIFT features
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
joblib.dump(knn, f'../artifacts/knn_model_sift_{height}_x_{width}')

# %%
y_pred = knn.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred)
y_pred = knn.predict(X_test)
print("test Shape:", X_test.shape)

test_accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print(f"Train Accuracy: {train_accuracy*100:.4f}%")
print(f"Test Accuracy: {test_accuracy*100:.4f}%")
print(f"F1 Score: {f1*100:.4f}%")
print(f"Recall: {recall*100:.4f}%")

test Shape: (5000, 128)
Train Accuracy: 1.0850%
Test Accuracy: 0.8800%
F1 Score: 0.0154%
Recall: 0.8800%


: 